# Tutorial 3: How simulations define your predictions
The inverse problem has no unique solution as it is ill-posed. In order to solve it we need to constraint the space of possible solutions. While inverse solutions like minimum-norm estimates have an explicit constraint of minimum-energy, the constraints with esinet are implicit and mostly shaped by the simulations.

This tutorial aims the relation between simulation parameters and predictions.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [21]:
info = get_info(sfreq=100)
fwd = create_forward_model(sampling="ico3", info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.8s remaining:    3.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.8s remaining:    1.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


# Extent

## Simulate

In [23]:
n_samples = 10000
settings = dict(duration_of_trial=0.20)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

Simulating data based on sparse patches.


100%|██████████| 10000/10000 [00:02<00:00, 4258.95it/s]


source data shape:  (1284, 20) (1284, 20)


100%|██████████| 10000/10000 [01:16<00:00, 130.30it/s]


## Create Data

In [28]:
import numpy as np
X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
y = np.squeeze(np.stack([src.data for src in sim.source_data]))
y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

X = np.swapaxes(X, 1,2)
y = np.swapaxes(y, 1,2)
print(X.shape, y.shape)

(10000, 20, 61) (10000, 20, 1284)


## Build and Train

In [46]:
import tensorflow.keras.backend as K
def sparsity(y_true, y_pred):
    return K.mean(K.square(y_pred)) / K.max(K.square(y_pred))
def custom_loss():
    def loss(y_true, y_pred):
        loss1 = tf.keras.losses.CosineSimilarity()(y_true, y_pred)
        loss2 = sparsity(None, y_pred)
        return loss1 + loss2 * 1e-3
    return loss


In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, TimeDistributed, Bidirectional, LSTM, GRU, multiply, Activation
from tensorflow.keras.regularizers import l1

leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 300
n_lstm_units = 30
activation_function = "tanh"
batch_size = 32
epochs = 30
dropout = 0.1

def threshold_activation(x):
    return tf.cast(x > 0.3, dtype=tf.float32)

inputs = tf.keras.Input(shape=(None, n_channels), name='Input')
fc1 = TimeDistributed(Dense(n_dense_units, 
            activation=activation_function), 
            name='FC1')(inputs)
direct_out = TimeDistributed(Dense(n_dipoles, 
            activation="linear"),
            name='FC2')(fc1)

lstm1 = Bidirectional(GRU(n_lstm_units, return_sequences=True, 
            input_shape=(None, n_dense_units), dropout=dropout), 
            name='LSTM1')(direct_out)
mask = TimeDistributed(Dense(n_dipoles, 
            activation="sigmoid"), 
            name='Mask')(lstm1)
mask = Activation(threshold_activation)(mask)

final_out = multiply([direct_out, mask], name="multiply")
model = tf.keras.Model(inputs=inputs, outputs=final_out, name='Contextualizer')


# model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer="adam")
model.compile(loss=custom_loss(), optimizer="adam", metrics=[tf.keras.losses.CosineSimilarity(), sparsity])

model.summary()
model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "Contextualizer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, None, 61)]   0                                            
__________________________________________________________________________________________________
FC1 (TimeDistributed)           (None, None, 300)    18600       Input[0][0]                      
__________________________________________________________________________________________________
FC2 (TimeDistributed)           (None, None, 1284)   386484      FC1[0][0]                        
__________________________________________________________________________________________________
LSTM1 (Bidirectional)           (None, None, 60)     236880      FC2[0][0]                        
_____________________________________________________________________________________

KeyboardInterrupt: 

# Eval

In [52]:
import numpy as np
n_samples = 2
settings = dict(duration_of_trial=0.25, number_of_sources=3)
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X_test = sim_test.eeg_data[0].average().data
X_test = (X_test- X_test.mean()) / X_test.std()
y_test = sim_test.source_data[0].data
y_test /= np.max(np.abs(y_test))

X_test = X_test[np.newaxis]
y_test = y_test[np.newaxis]

X_test = np.swapaxes(X_test, 1,2)
y_test = np.swapaxes(y_test, 1,2)
print(X_test.shape, y_test.shape)

y_hat = model.predict(X_test)[0]

stc = sim_test.source_data[0]
stc.plot(**plot_params)


stc_hat = stc.copy()
stc_hat.data = y_hat.T
stc_hat.plot(**plot_params)


Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 665.29it/s]


source data shape:  (1284, 25) (1284, 25)


100%|██████████| 2/2 [00:00<00:00, 99.95it/s]


(1, 25, 61) (1, 25, 1284)
